In [49]:
import pandas as pd

In [50]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

In [51]:
column = 'по должности-лемме'

In [52]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
#for i in 'работа финансовым директором'.split():
#    print(morph.normal_forms(i))
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")

In [53]:
def morphe(str):
    return ' '.join([morph.normal_forms(w)[0] for w in str.split() if w not in stopwords_ru])

In [54]:
df = pd.read_csv("answers.csv")
df = df[["query", column]]
#df = df[df["по должности-лемме"].notna()]
df.fillna({column: 'Нет'}, inplace=True)

df['stem_query'] = df['query'].apply(morphe)
df

,query,по должности-лемме,stem_query
0,фарпост работа владивосток,Нет,фарпост работа владивосток
1,кофейни вакансии,Нет,кофейня вакансия
2,работа разнорабочие часовой,Рабочий,работа разнорабочий часовой
3,личный водитель на день,Водитель,личный водитель день
4,работа от работодателя персональный водитель,Водитель,работа работодатель персональный водитель
...,...,...,...
14240,япония вакансии,Нет,япония вакансия
14241,япония работа,Нет,япония работа
14242,японский язык работа,Нет,японский язык работа
14243,яппи вакансии,Нет,яппи вакансия


In [55]:
from sklearn.model_selection import train_test_split

sentences = df['stem_query'].values
y = df[column].values

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

query_train, query_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences)

X_train = vectorizer.transform(query_train)
X_test  = vectorizer.transform(query_test)

In [57]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.8966872543514879


In [72]:
def predict_position(query):
    query = morphe(query)
    query_transformed = vectorizer.transform([query])
    prediction = classifier.predict(query_transformed)
    return encoder.inverse_transform(prediction)[0]

example_query = "программист"
predicted_position = predict_position(example_query)
print(f"Предсказание {column}:", predicted_position)

Предсказание по должности-лемме: Программист
